In [1]:
import tensorflow

2024-04-15 13:43:53.355523: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-15 13:43:53.357269: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-15 13:43:53.379676: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-04-15 13:43:53.379710: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-04-15 13:43:53.380244: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to

In [2]:
import hls4ml

WARN: Unable to import optimizer(s) from expr_templates.py: No module named 'sympy'


/home/formal/miniconda3/envs/hls_har/lib/python3.10/site-packages/hls4ml/converters/__init__.py:27: UserWarning: WARNING: Pytorch converter is not enabled!
  warnings.warn("WARNING: Pytorch converter is not enabled!", stacklevel=1)


In [3]:
from tensorflow import keras


In [4]:
pip show tensorflow

Name: tensorflow
Version: 2.15.0
Summary: TensorFlow is an open source machine learning framework for everyone.
Home-page: https://www.tensorflow.org/
Author: Google Inc.
Author-email: packages@tensorflow.org
License: Apache 2.0
Location: /home/formal/miniconda3/envs/hls_har/lib/python3.10/site-packages
Requires: absl-py, astunparse, flatbuffers, gast, google-pasta, grpcio, h5py, keras, libclang, ml-dtypes, numpy, opt-einsum, packaging, protobuf, setuptools, six, tensorboard, tensorflow-estimator, tensorflow-io-gcs-filesystem, termcolor, typing-extensions, wrapt
Required-by: hls4ml
Note: you may need to restart the kernel to use updated packages.


In [5]:
import tensorflow as tf

In [6]:
from keras.models import load_model
model = load_model('recognizer.h5')
# model = load_model('KERAS_check_best_model.h5')


In [7]:
config = hls4ml.utils.config_from_keras_model(model, granularity='name')

for Layer in config['LayerName'].keys():
    config['LayerName'][Layer]['Precision'] = 'ap_fixed<8,2>'   #balancing between acuracy and resource usage
    config['LayerName'][Layer]['Strategy'] = 'resource'
    config['LayerName'][Layer]['ReuseFactor'] = 15
#  how many times the resources within a layer can be reused during computation

Interpreting Sequential
Topology:
Layer name: dense_7_input, layer type: InputLayer, input shapes: [[None, 128]], output shape: [None, 128]
Layer name: dense_7, layer type: Dense, input shapes: [[None, 128]], output shape: [None, 128]
Layer name: dense_8, layer type: Dense, input shapes: [[None, 128]], output shape: [None, 64]
Layer name: dense_9, layer type: Dense, input shapes: [[None, 64]], output shape: [None, 64]
Layer name: dense_10, layer type: Dense, input shapes: [[None, 64]], output shape: [None, 32]
Layer name: dense_11, layer type: Dense, input shapes: [[None, 32]], output shape: [None, 32]
Layer name: dense_12, layer type: Dense, input shapes: [[None, 32]], output shape: [None, 5]


In [26]:
# config = hls4ml.utils.config_from_keras_model(model,granularity = 'Model')


In [27]:
# config['Model']['Precision'] = 'ap_fixed<3,2>' #reduces precision
# config['Model']['Strategy'] = 'resource' #changes strategy to resource
# config['Model']['ReuseFactor'] = 1



In [12]:
ls_model = hls4ml.converters.convert_from_keras_model(model,
                                                       hls_config = config,
                                                       io_type = 'io_stream',
                                                       output_dir = 'hlsProjectCreated2')

Interpreting Sequential
Topology:
Layer name: dense_7_input, layer type: InputLayer, input shapes: [[None, 128]], output shape: [None, 128]
Layer name: dense_7, layer type: Dense, input shapes: [[None, 128]], output shape: [None, 128]
Layer name: dense_8, layer type: Dense, input shapes: [[None, 128]], output shape: [None, 64]
Layer name: dense_9, layer type: Dense, input shapes: [[None, 64]], output shape: [None, 64]
Layer name: dense_10, layer type: Dense, input shapes: [[None, 64]], output shape: [None, 32]
Layer name: dense_11, layer type: Dense, input shapes: [[None, 32]], output shape: [None, 32]
Layer name: dense_12, layer type: Dense, input shapes: [[None, 32]], output shape: [None, 5]
Creating HLS model


In [29]:
# ls_model.write()

In [13]:
import os 

In [14]:
os.environ['PATH'] = '/home/formal/Vivado/2019.2/bin:' + os.environ['PATH']

In [15]:
ls_model.compile()

Writing HLS project
Done


In [16]:
ls_model.build(csim=True)


****** Vivado(TM) HLS - High-Level Synthesis from C, C++ and SystemC v2019.2 (64-bit)
  **** SW Build 2708876 on Wed Nov  6 21:39:14 MST 2019
  **** IP Build 2700528 on Thu Nov  7 00:09:20 MST 2019
    ** Copyright 1986-2019 Xilinx, Inc. All Rights Reserved.

source /home/formal/Vivado/2019.2/scripts/vivado_hls/hls.tcl -notrace
INFO: [HLS 200-10] Running '/home/formal/Vivado/2019.2/bin/unwrapped/lnx64.o/vivado_hls'
INFO: [HLS 200-10] For user 'formal' on host 'formal-HP-Z2-G8-Tower-Workstation-Desktop-PC' (Linux_x86_64 version 5.15.0-101-generic) on Mon Apr 15 13:45:33 IST 2024
INFO: [HLS 200-10] On os Ubuntu 20.04.6 LTS
INFO: [HLS 200-10] In directory '/home/formal/hlsProjectCreated2'
Sourcing Tcl script 'build_prj.tcl'
INFO: [HLS 200-10] Creating and opening project '/home/formal/hlsProjectCreated2/myproject_prj'.
INFO: [HLS 200-10] Adding design file 'firmware/myproject.cpp' to the project
INFO: [HLS 200-10] Adding test bench file 'myproject_test.cpp' to the project
INFO: [HLS 200-

{'CSimResults': [['0.234375', '0.203125', '0.234375', '0.203125', '0.25']],
 'CSynthesisReport': {'TargetClockPeriod': '5.00',
  'EstimatedClockPeriod': '4.316',
  'BestLatency': '36',
  'WorstLatency': '37',
  'IntervalMin': '19',
  'IntervalMax': '20',
  'BRAM_18K': '448',
  'DSP': '1',
  'FF': '31851',
  'LUT': '468331',
  'URAM': '0',
  'AvailableBRAM_18K': '5376',
  'AvailableDSP': '12288',
  'AvailableFF': '3456000',
  'AvailableLUT': '1728000',
  'AvailableURAM': '1280'}}

In [19]:
hls4ml.report.read_vivado_report('hlsProjectCreated2')

Found 1 solution(s) in hlsProjectCreated2/myproject_prj.
Reports for solution "solution1":

C SIMULATION RESULT:
INFO: [SIM 2] *************** CSIM start ***************
INFO: [SIM 4] CSIM will launch GCC as the compiler.
   Compiling ../../../../myproject_test.cpp in debug mode
   Compiling ../../../../firmware/myproject.cpp in debug mode
   Generating csim.exe
INFO: Unable to open input/predictions file, using default input.
0.234375 0.203125 0.234375 0.203125 0.25 
INFO: Saved inference results to file: tb_data/csim_results.log
INFO: [SIM 1] CSim done with 0 errors.
INFO: [SIM 3] *************** CSIM finish ***************

SYNTHESIS REPORT:
== Vivado HLS Report for 'myproject'
* Date:           Mon Apr 15 16:23:38 2024

* Version:        2019.2 (Build 2704478 on Wed Nov 06 22:10:23 MST 2019)
* Project:        myproject_prj
* Solution:       solution1
* Product family: virtexuplus
* Target device:  xcvu13p-flga2577-2-e


== Performance Estimates
+ Timing: 
    * Summary: 
    +-----